In [1]:
import pathlib
import json
import os
import sqlite3
import sys
import time
from datetime import datetime
import urllib.request
import csv

In [ ]:
# 1FZ8-FyIyxihPpYMlkkZ4i6muQP8VTDJyWzDnM4CWQpU - sspreadsheet_id

In [21]:
AIMLAB_DB_PATH = os.path.abspath(os.path.join(os.getenv("APPDATA"), os.pardir, "LocalLow\\statespace\\aimlab_tb\\klutch.bytes"))
AIMLAB_DB_PATH

'C:\\Users\\logan\\AppData\\LocalLow\\statespace\\aimlab_tb\\klutch.bytes'

In [24]:

for csid, name in cs_level_ids.items():
    print(csid, name)

CsLevel.VT Empyrean.VT Angle.RB668Z vt angleshot novice
CsLevel.Lowgravity56.VT x WHJ.RWEZ9J vt x whj smooth strafe sphere
CsLevel.Lowgravity56.VT Adjus.ROJETY vt adjust track valorant
CsLevel.Lowgravity56.VT 3T Wi.RTA5MX vt 3t wide
CsLevel.Lowgravity56.VT Berry.RUPUHP vt berryts static small
CsLevel.Lowgravity56.VT x WHJ.RWFB5F vt x whj 5 sphere hipfire small
CsLevel.VT Empyrean.VT 1w2ts.R21FUT vt 1w2ts smallflicks valorant
CsLevel.Lowgravity56.VT Dynam.RQCD1Z vt dynamic reflex micro
CsLevel.Lowgravity56.VT x WHJ.RWFQU4 vt x whj speedswitch click


In [29]:
# Open db connection
con = sqlite3.connect(AIMLAB_DB_PATH)
cur = con.cursor()

# Get scores from the database
result = []
for csid, name in cs_level_ids.items():
    # Find new tasks that occured past ended at
    cur.execute(
        f"SELECT taskName, score, endedAt FROM TaskData WHERE taskName = ? AND createDate > date(?)",
        [csid, blacklist[name]])
    temp = cur.fetchall()
    result.extend(temp)
result

[('CsLevel.VT Empyrean.VT Angle.RB668Z', 566, '2023-12-31 04:57:02'),
 ('CsLevel.VT Empyrean.VT Angle.RB668Z', 371, '2024-01-22 21:00:04'),
 ('CsLevel.Lowgravity56.VT x WHJ.RWEZ9J', 1763, '2023-12-30 23:13:11'),
 ('CsLevel.Lowgravity56.VT x WHJ.RWEZ9J', 1533, '2023-12-30 23:15:44'),
 ('CsLevel.Lowgravity56.VT x WHJ.RWEZ9J', 1850, '2023-12-30 23:17:02'),
 ('CsLevel.Lowgravity56.VT x WHJ.RWEZ9J', 1746, '2024-01-02 20:19:56'),
 ('CsLevel.Lowgravity56.VT x WHJ.RWEZ9J', 2068, '2024-01-02 20:21:07'),
 ('CsLevel.Lowgravity56.VT x WHJ.RWEZ9J', 1928, '2024-01-02 20:22:18'),
 ('CsLevel.Lowgravity56.VT x WHJ.RWEZ9J', 1953, '2024-01-03 20:51:03'),
 ('CsLevel.Lowgravity56.VT x WHJ.RWEZ9J', 2136, '2024-01-03 20:52:13'),
 ('CsLevel.Lowgravity56.VT x WHJ.RWEZ9J', 2095, '2024-01-03 20:53:22'),
 ('CsLevel.Lowgravity56.VT x WHJ.RWEZ9J', 1753, '2024-01-04 22:20:55'),
 ('CsLevel.Lowgravity56.VT x WHJ.RWEZ9J', 1880, '2024-01-04 22:22:08'),
 ('CsLevel.Lowgravity56.VT x WHJ.RWEZ9J', 1727, '2024-01-04 22:23:18

In [ ]:
# 371
# VT Angleshot Novice

In [19]:
file_path = 'data.csv'
cs_level_ids = dict()
blacklist = dict()

# Open and read the CSV file
with open(file_path, newline='', encoding='utf-8') as csvfile:
    # Create a CSV reader
    csv_reader = csv.reader(csvfile)

    # Skip the header row
    next(csv_reader)

    # Iterate through the lines
    for splits in csv_reader:
        name = splits[0].replace('"', '')
        cs_level_id = splits[1].replace('"', '')
        cs_level_ids[cs_level_id] = name.lower()
        date = datetime.strptime(splits[2].replace('"', '').replace('\n', ''), "%d.%m.%Y").date()
        blacklist[name.lower()] = date

cs_level_ids

{'CsLevel.VT Empyrean.VT Angle.RB668Z': 'vt angleshot novice',
 'CsLevel.Lowgravity56.VT x WHJ.RWEZ9J': 'vt x whj smooth strafe sphere',
 'CsLevel.Lowgravity56.VT Adjus.ROJETY': 'vt adjust track valorant',
 'CsLevel.Lowgravity56.VT 3T Wi.RTA5MX': 'vt 3t wide',
 'CsLevel.Lowgravity56.VT Berry.RUPUHP': 'vt berryts static small',
 'CsLevel.Lowgravity56.VT x WHJ.RWFB5F': 'vt x whj 5 sphere hipfire small',
 'CsLevel.VT Empyrean.VT 1w2ts.R21FUT': 'vt 1w2ts smallflicks valorant',
 'CsLevel.Lowgravity56.VT Dynam.RQCD1Z': 'vt dynamic reflex micro',
 'CsLevel.Lowgravity56.VT x WHJ.RWFQU4': 'vt x whj speedswitch click'}

In [15]:
def init_cs_level_ids_and_blacklist() -> (dict, dict):
    file_path = 'data.csv'
    cs_level_ids = dict()
    blacklist = dict()

    # Open and read the CSV file
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        # Create a CSV reader
        csv_reader = csv.reader(csvfile)

        # Skip the header row
        next(csv_reader)

        # Iterate through the lines
        for splits in csv_reader:
            name = splits[0].replace('"', '')
            cs_level_id = splits[1].replace('"', '')
            cs_level_ids[cs_level_id] = name.lower()
            date = datetime.strptime(splits[2].replace('"', '').replace('\n', ''), "%d.%m.%Y").date()
            blacklist[name.lower()] = date

    return cs_level_ids, blacklist

cs_level_ids, blacklist = init_cs_level_ids_and_blacklist()
cs_level_ids

{'CsLevel.VT Empyrean.VT Angle.RB668Z': 'vt angleshot novice',
 'CsLevel.VT Empyrean.VT Waves.R8TGLS': 'vt waveshot novice',
 'CsLevel.VT Empyrean.VT Sixsh.R8TGD3': 'vt sixshot novice',
 'CsLevel.VT Empyrean.VT Multi.RB6AAB': 'vt multishot 90 novice',
 'CsLevel.canner.VT Suave.R8I6UD': 'vt suavetrack novice',
 'CsLevel.VT Empyrean.VT Stead.RB67MZ': 'vt steadytrack novice',
 'CsLevel.zcr.VT Pillt.R8KMR0': 'vt pilltrack novice',
 'CsLevel.Lowgravity56.VT Axitr.RJF5AX': 'vt axitrack novice',
 'CsLevel.VT Empyrean.VT Spher.RB69A4': 'vt sphereswitch novice',
 'CsLevel.VT Empyrean.VT Skysw.R8TH9B': 'vt skyswitch novice',
 'CsLevel.VT Empyrean.VT Angle.R8X0YY': 'vt angleswitch novice',
 'CsLevel.VT Empyrean.VT Arcsw.RB6A0U': 'vt arcswitch novice',
 'CsLevel.VT Empyrean.VT Angle.R8S2XN': 'vt angleshot intermediate',
 'CsLevel.VT Empyrean.VT Waves.R8S37J': 'vt waveshot intermediate',
 'CsLevel.VT Empyrean.VT Fives.R8YV6C': 'vt fiveshot intermediate',
 'CsLevel.VT Empyrean.VT Multi.R8S3PL': 'vt 